In [ ]:
import random
import os
import sys 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn import linear_model
import gzip
import scipy.interpolate as si
from Bio import Phylo
import networkx as nx
#import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout
#import ndex2.client
#import ndex2
import os
import sys
import time
import collections
random.seed(0)

In [ ]:
mpl.rcParams['font.family']       = 'Helvetica'
mpl.rcParams['font.sans-serif']   = ["Helvetica","Arial","DejaVu Sans","Lucida Grande","Verdana"]
mpl.rcParams['figure.figsize']    = [4,3]
mpl.rcParams['font.size']         = 9
mpl.rcParams["axes.labelcolor"]   = "#000000"
mpl.rcParams["axes.linewidth"]    = 1.0 
mpl.rcParams["xtick.major.width"] = 1.0
mpl.rcParams["ytick.major.width"] = 1.0
cmap1 = plt.cm.tab10
cmap2 = plt.cm.Set3  
colors1 = [cmap1(i) for i in range(0,10)]
colors2 = [cmap2(i) for i in range(0,12)] 
plt.style.use('default')

In [ ]:
os.chdir("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0142")
try:
    os.mkdir("figures")
except:
    None

In [ ]:
# Runtime fitting
df_runtime = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0142/fitting/runtime_result.gc.csv", names = ["Nseq","Runtime","RuntimeUnit","Niter","unit"])
#df_runtime

In [ ]:
df_runtime_ext = df_runtime[(df_runtime["Niter"]==1) & (df_runtime["Nseq"]>10000)]
clf = linear_model.LinearRegression(fit_intercept=False) # 原点を通るモデル

X = [[x] for x in df_runtime_ext["Nseq"]/1000]
Y=df_runtime_ext["Runtime"]
clf.fit(X, Y)
# Y = aX
# a
print("y = ", clf.coef_[0]/1000, "x,", "R^2 = ", clf.score(X, Y))

fig = plt.figure(figsize=(2,2))
ax = fig.add_axes([0.1,0.1,0.8,0.8])


ax.scatter(df_runtime_ext["Nseq"]/1000, df_runtime_ext["Runtime"], color = "#7F33FF", alpha = 0.5, s=15)

plt.plot(X, clf.predict(X), color = "#000000")

ax.set_xticks([0,200,400])
ax.set_xlabel("Number of sequences\nin each FRACTAL iteration ($\\times 10^3$)")
ax.set_ylabel("Runtime (sec)")
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.savefig("figures/NK_0142_runtime.pdf", bbox_inches='tight')
plt.close()

In [ ]:
df=pd.read_csv('/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0142/simulation/result.runtime.sim.csv', names=['ID','tip_num', 'node_num', 'model', 'time', 'time_unit'])

def fmt(X, Y, Z, x, y):
    # input: X,Y: x,yのmeshgrid, Z: f(X,Y)の値, x,y:求めたいzのx,y座標
    # output: f(x,y)の値
    z = np.take(si.interp2d(X, Y, Z)(x, y), 0)
    return z

tip_num_range=np.array([1048576,2097152,4194304,8388608,16777216,33554432,67108864,134217728,268435456,536870912,1073741824,2147483648,4294967296,8589934592,17179869184])
node_num_range=np.array([1,2,5,10,20,30,40,50,60,70,80,90,100, 150, 200, 300, 400, 600, 800, 1200, 1600, 2400, 3200, 4800, 6400, 10000, 15000, 20000])

plt.style.use('dark_background')
y = tip_num_range
x = node_num_range
X, Y = np.meshgrid(x, np.log10(y))


for model in [1,2]:
    Z=np.zeros((len(y),len(x)))

    ext_df=df[df.model==model]
    for i in range(len(ext_df)):
        Z[list(y).index(list(ext_df.tip_num)[i])][list(x).index(list(ext_df.node_num)[i])]=list(ext_df.time)[i]

    Z_small=Z
    for i in range(len(Z_small)):
        for j in range(len(Z_small[0])):
            Z_small[i][j]=np.log10(Z_small[i][j]/86400)

    fig = plt.figure(figsize=(1.6,1.6))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])

    #set_levels
    levels=[-4,-3,-2,-1,0,1,2, 3,4]
    #num_of_levels=int(input())
    #for i in range(num_of_levels):
    #    levels.append(float(input()))

    cont = ax.contour(X, Y, Z_small, levels=levels,colors=['white'])
    cont.clabel(fmt='$10^{%1.0f}$', fontsize=10)
    ax.tick_params(labelsize=10)
    ax.set_facecolor((0,0,0,1))
    cf=plt.contourf(X,Y,Z_small, cmap='jet', levels=np.arange(-5,4.5,0.1))

    # get estimated at Fig. 2 condition
    Nnodes=300
    Nseq=235100752
    log10_days=fmt(np.log10(X),Y,Z_small, np.log10(Nnodes), np.log10(Nseq))
    days=pow(10,log10_days)
    hrs=days*24
    print("model: "+str(model)+"\nNum. of nodes: "+str(Nnodes)+"\nNum. of sequences: "+str(Nseq)+"\nEstimated Run Time: "+str(hrs)+" hrs")


    #plt.colorbar(cf,ax=ax,ticks=[-4,-3,-2,-1,0,1,2,3,4])
    ax.set_xscale("log")
    ax.set_ylim(6,10.2)

    #ax.zaxis.set_major_locator(LinearLocator(10))
    #ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    ax.set_xlabel("#computer nodes used")
    ax.set_ylabel("Number of sequences\n($10^x$ sequences)")
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    fig.savefig("figures/NK_0142_simulated_time_mode"+str(model)+"_dark.pdf",bbox_inches="tight")
    #plt.show()
    plt.close()

In [ ]:
df_fractal = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0142/accuracy/accuracy.fractal.csv",names=["Nseq","Ntips","NRFD"])
df_rapidnj = pd.read_csv("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0142/accuracy/accuracy.rapidnj.csv",names=["Nseq","Ntips","NRFD"])
df_fractal["Accuracy"] = (1-df_fractal["NRFD"])*100
df_rapidnj["Accuracy"] = (1-df_rapidnj["NRFD"])*100
df_fractal

In [ ]:
plt.style.use('dark_background')

fig=plt.figure(figsize=(1.2,1.2))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
ax.scatter(x = df_fractal['Ntips'], y = df_fractal["Accuracy"], color ="#7F33FF",s=8)
ax.set_xlabel("")
ax.set_xscale("log")

ax.set_ylim(99.6,100)
ax.set_yticks([99.6,99.7,99.8,99.9,100])
ax.set_xticks(list(np.arange(1,11)*100000))

#ax.set_xticklabels([100000,1000000])
ax.set_xlim(100000,1100000)

ax.tick_params(left=True, labelleft=True,right=False, labelright=False, bottom=True, labelbottom=False, top=False, labeltop=False)

fig.savefig("figures/NK_0142_fractal_accuracy.pdf",bbox_inches="tight")
plt.show()
plt.close()
fig=plt.figure(figsize=(3, 2.5))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
ax.scatter(x = df_fractal['Ntips'], y = df_fractal["Accuracy"], color ="#7F33FF",s=8)
ax.scatter(x = df_rapidnj['Ntips'], y = df_rapidnj["Accuracy"], color ="#FFD479",s=8)
ax.set_xlabel("Number of sequences")
ax.set_ylabel("1- NRFD (%)")
ax.set_ylim(0,105)
ax.set_xlim(500,1.2*(np.power(10,6)))
ax.set_xscale("log")
ax.tick_params(axis = 'x', colors ='white', labelsize=9)
ax.tick_params(axis = 'y', colors = 'white', labelsize=9)
ax.spines["top"].set_color("none")
ax.spines["right"].set_color("none")
ax.set_facecolor("none")
fig.savefig("figures/NK_0142_fractal_rapidnj_accuracy.pdf",bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig=plt.figure(figsize=(3, 2.5))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
ax.scatter(x = df_fractal['Ntips'], y = df_fractal["Accuracy"], color ="#7F33FF",s=8)
ax.scatter(x = df_rapidnj['Ntips'], y = df_rapidnj["Accuracy"], color ="#FFD479",s=8)
ax.set_xlabel("Number of sequences")
ax.set_ylabel("1- NRFD (%)")
ax.set_ylim(98,100)
ax.set_xlim(800,1.2*(np.power(10,6)))
ax.set_xscale("log")
ax.tick_params(axis = 'x', colors ='white', labelsize=9)
ax.tick_params(axis = 'y', colors = 'white', labelsize=9)
ax.spines["top"].set_color("none")
ax.spines["right"].set_color("none")
ax.set_facecolor("none")
plt.show()
plt.close()

In [ ]:
df_fractal[df_fractal['Nseq'] == 100000]

In [ ]:
df_time = pd.read_table("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0142/time/node_count.txt", names = ('date', 'date_sec', 'running', 'running+waiting'))

In [ ]:
df_time['waiting'] = df_time['running+waiting'] - df_time['running']
df_time['run_time'] = df_time['date_sec'] - 1607930353
df_time

In [ ]:
fig = plt.figure(figsize=(2,1.4))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
#sns.lineplot(data=df, x="run_time", y="running+waiting", label = 'running + waiting')
#sns.lineplot(data=df, x="run_time", y="running",label='running')

x=list(df_time['run_time'])

plt.stackplot(x,df_time["running+waiting"], color = "#6E8A92", alpha=1 )
plt.stackplot(x,df_time['running'], color = "#576C74", alpha=1 )

ax.set_xlim(0,120000)
ax.set_xlabel("Time (sec)")
ax.set_ylabel("Number of computing\nnodes used")
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.savefig("figures/NK_0142_time_node.pdf", bbox_inches='tight')
plt.close()

In [ ]:
def ask_user_info():
    print("Enter your account:")
    my_account = input()
    print("Enter your password:")
    my_password = input()
    return my_account, my_password

server="http://test.ndexbio.org"
my_account, my_password = ask_user_info()
my_ndex=ndex2.client.Ndex2(server, my_account, my_password)

In [ ]:
# get info of ndex network set
set_dict = my_ndex.get_network_set("984166e6-4001-11eb-8821-0660b7976219")

In [ ]:
with open("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0142/hiview/uuid_dir_name.txt", 'w') as handle:
    for network_uuid in set_dict['networks']:
        #network_uuid    = set_dict['networks'][0]
        network_summary = my_ndex.get_network_summary(network_uuid)
        print(network_summary['externalId'], network_summary['description'], network_summary['name'], sep = '\t', file = handle)

In [ ]:
%ls -l /Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0142/hiview/uuid_dir_name.txt

In [ ]:
tree = Phylo.read("/Users/nk/Documents/backupped/Research/YachieLabLocal/FRACTAL/data/NK_0142/hiview/FRACTALclustering_history.2.nwk", 'newick')
node2depth = tree.depths()

In [ ]:
plt.hist([node2depth[tip] for tip in tree.get_terminals()], range= (0,10), bins= 10)

In [ ]:
np.median([node2depth[tip] for tip in tree.get_terminals()])

In [ ]:
len(tree.get_nonterminals() + tree.get_terminals())